# Income/Expense Sankey Visualization

## Miniconda Setup
```
conda env create --file environment.yml
conda activate income-expense-sankey
```

## Imports

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

## CSV Import

In [ ]:
csv_path = 'C:/Users/andri/Nextcloud/Dokumente/Steuern/2024/usa_transactions.csv'

transactions_frame = pd.read_csv(csv_path, delimiter=';')

transactions_frame['betrag'] = transactions_frame['betrag'].str.replace('CHF ', '')
transactions_frame['betrag'] = transactions_frame['betrag'].str.replace('’', '')
transactions_frame['betrag'] = transactions_frame['betrag'].astype(float)

## Data Processing

In [ ]:
income_transactions_frame = transactions_frame[transactions_frame['betrag'] > 0]
expense_transactions_frame = transactions_frame[transactions_frame['betrag'] < 0]
expense_transactions_frame.loc[:, 'betrag'] = expense_transactions_frame['betrag'].abs()

source_arr = []
target_arr = []
value_arr = []
label_arr = []

income_categories = income_transactions_frame['category'].unique()
expense_categories = expense_transactions_frame['category'].unique()
expense_subcategories = expense_transactions_frame['subcategory'].unique()
label_arr = np.concatenate((["total_income", "total_expense"], income_categories, expense_categories, expense_subcategories))

source_arr.append(0)
target_arr.append(1)
value_arr.append(expense_transactions_frame['betrag'].sum())

for i in range(2, len(income_categories)+2):
    source_arr.append(i)
    target_arr.append(0)
for income_category in income_categories:
    value_arr.append(income_transactions_frame[income_transactions_frame['category'] == income_category]['betrag'].sum())

for i in range(len(income_categories)+2, len(income_categories)+len(expense_categories)+2):
    source_arr.append(1)
    target_arr.append(i)
for expense_category in expense_categories:
    value_arr.append(expense_transactions_frame[expense_transactions_frame['category'] == expense_category]['betrag'].sum())

for expense_category in expense_categories:
    for expense_subcategory in expense_subcategories:
        subcategory_sum = expense_transactions_frame[(expense_transactions_frame['category'] == expense_category) & (expense_transactions_frame['subcategory'] == expense_subcategory)]['betrag'].sum()
        if subcategory_sum > 0:
            source_arr.append(np.where(label_arr == expense_category)[0][0])
            target_arr.append(np.where(label_arr == expense_subcategory)[0][-1])
            value_arr.append(subcategory_sum)

## Data Visualization

In [ ]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 3,
      label = label_arr,
    ),
    link = dict(
      source = source_arr,
      target = target_arr,
      value = value_arr
  ))])

fig.update_layout(title_text="Income/Expense Sankey Diagram", font_size=10)
fig.show()